In [1]:
from ipywidgets import HBox, VBox, Layout
from IPython.display import display
import ipywidgets as widgets
import plotly.express as px
import pandas as pd

# convertir la hoja excel a un dataframe utilizando pandas
data=pd.read_excel('data_jup_01.xlsx')


In [2]:
# Función para filtrar los datos y graficar los rendimientos
def filtros_graficos(level_option, subjets_option, area_option, expand_option,select_date):
       
        asignatura = subjets_option # Nombre de la asignatura elegida para el título del gráfico
        data['PERIODO'] = pd.to_datetime(data['PERIODO']) #convierte la columna 'PERIODO' al formato de tiempo
        data['month'] = data['PERIODO'].dt.month #agrega una columna numérica al dataframe, correspondiente al mes

        months = select_date # variable para alamcenar lista de meses seleccionados
        number_months = len(months) # cantidad de meses para ciclo for de las etiquetas de los sub_gráficos
        calendario = {1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril", 5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
                      9: "Septiembre", 10: "Octubre", 11: "Noviembre",12: "Diciembre" }
    
# data frame filtrado segun mes o meses elegidos
        data_f_months= data[data['month'].isin(months)]

    
        if area_option =='PLAN COMÚN':

            if expand_option == 1: # Opcion para expandir Nivel, grafica los cursos por separado
                filtered_data=data_f_months[(data_f_months['NIVEL']==level_option) & (data_f_months['ASIGNATURA']==subjets_option)] # Datos por cada curso
                final_data = filtered_data
                graph_x_axes = 'CURSO'

            elif expand_option == 2: # Opcion para contraer Nivel, grafica el Nivel completo
                filtered_data=data_f_months[(data_f_months['NIVEL']==level_option) & (data_f_months['ASIGNATURA']==subjets_option)] # Datos por cada curso
                mean_data = filtered_data.groupby(['PERIODO','NIVEL','ASIGNATURA']).mean(numeric_only=True).reset_index() # Datos Promediados por Nivel
                final_data = mean_data
                graph_x_axes = 'NIVEL'

        elif area_option =='CARRERAS' or area_option =='PROFUNDIZACIÓN HC' :

            if expand_option == 1: # Opcion para expandir Nivel, grafica los cursos por separado
                filtered_data=data_f_months[(data_f_months['NIVEL']==level_option) & (data_f_months['TIPO']==subjets_option)] # Datos por cada modulo o electivo
                final_data = filtered_data
                graph_x_axes = 'ASIGNATURA'
            
            elif expand_option == 2: # Opcion para contraer Nivel, grafica el Nivel completo
                filtered_data=data_f_months[(data_f_months['NIVEL']==level_option) & (data_f_months['TIPO']==subjets_option)] # Datos por cada modulo o electivo
                mean_data = filtered_data.groupby(['PERIODO','NIVEL','TIPO']).mean(numeric_only=True).reset_index() # Datos Promediados por nivel
                final_data = mean_data
                graph_x_axes = 'NIVEL'
                
        trace01 = px.bar(final_data, x=graph_x_axes, y=['MB','B','S','I','P'],
                     title= f'RENDIMIENTO ESTUDIANTES en {asignatura}',
                     facet_col='PERIODO', #Dibujar uno o más gráficos separados por fecha, en este caso el mes de recolección de los datos
                     #width=1024, height=440,
                     labels={'value':'','variable':'Categorías','CURSO':'Cursos'},
                     barmode='group',
                     color_discrete_map={'MB':'blue','B':'green','S':'gold','I':'red','P':'silver'},
                     template="simple_white",
                     range_y=[0,1],
                     facet_col_spacing=0.08
                     )
    
        for i in range(number_months): # Itera los titulos de los sub_graficos segun el mes o meses elegidos
              trace01.layout.annotations[i].update(text=calendario[months[i]])

            
        trace01.update_yaxes(tickformat=".1%", tickfont_weight='bold',title_font_weight='bold',tickfont_size=15)
        trace01.update_xaxes(tickfont_weight='bold', title_font_weight='bold')
        trace01.update_layout(
                         hoverlabel_font_color='white',
                         hoverlabel_font_family='Consolas',
                         uniformtext_minsize=5,
                         uniformtext_mode='show',
                         title_font_weight='bold',
                         font_family='Consolas',
                         title_font_size=20,
                         title_x=0.5
                         )
        trace01.update_yaxes(showticklabels=True)
        
   
        trace01.show()
       
        #print(f"You selected: {level_option}")
        #print(months)

In [3]:
# Controles interactivos, botones y listas despegables independientes y dependientes

# Seleccionar el o los períodos para graficar (posibilidad de selección múltiple)
select_date_multiple=widgets.SelectMultiple(
    options=[('ABRIL',4), ('JUNIO',6)],
    value =[4],
    rows=4,
    description='PERIODO',
    disabled=False
)

# Selector para elegir entre el gráfico promedio de un nivel o el grafico con los cursos por separado.
expand_buttons=widgets.RadioButtons(
    options=[('Expandir',1), ('Contraer',2)],
#   value='Expandir', # Valor por defecto 'Expandir'
    layout={'width': 'max-content'}, # Si los nombres de los items son muy largos
    description='Expandir o Contraer Nivel:',
    disabled=False
)

# Lista despegable para áreas
data_cleaned = data.dropna(subset=['AREA'])
area_dropdown = widgets.Dropdown(
    options=data_cleaned['AREA'].unique().tolist(),
    description='Area:',
    disabled=True,
)

# Primera lista despegable para niveles 
grade_dropdown = widgets.Dropdown(
    options=data['NIVEL'].unique().tolist(),
    description='Nivel:',
    disabled=False,
)

# Segunda lista despegable para asignaturas (dependiente de la lista despegable niveles)
subjets_dropdown =widgets.Dropdown(
    options=data[data['NIVEL'] == '1MEDIO']['ASIGNATURA'].unique().tolist(),  # opcion por defecto es'1MEDIO'
    description='Asignaturas:',
    disabled=False,
)

# Función para actualizar las asignaturas basadas en la selección del grado y área
def update_subjets_dropdown(*args):
    
    selected_grade = grade_dropdown.value
    selected_area  = area_dropdown.value
    
    if (selected_grade =='2MEDIO' or selected_grade=='1MEDIO') and (
        selected_area =='CARRERAS' or selected_area=='PROFUNDIZACIÓN HC'):

        area_dropdown.value ='PLAN COMÚN' # Valor inicial del área cuando Nivel elegido es 1° Medio o 2° Medio

    # Fixed indentation from here
    elif selected_grade == '3MEDIO' or selected_grade =='4MEDIO':
        
        area_dropdown.disabled=False
        
        if selected_area =='PLAN COMÚN':
            filtered_subjets = data[(data['NIVEL'] == selected_grade) & (data['AREA'] == selected_area) ]['ASIGNATURA'].unique().tolist()
            subjets_dropdown.options = filtered_subjets

        elif selected_area =='CARRERAS':
            filtered_subjets = data[(data['NIVEL'] == selected_grade) & (data['AREA'] == selected_area) ]['TIPO'].unique().tolist()
            subjets_dropdown.options = filtered_subjets

        elif selected_area =='PROFUNDIZACIÓN HC':
            filtered_subjets = data[(data['NIVEL'] == selected_grade) & (data['AREA'] == selected_area) ]['TIPO'].unique().tolist()
            subjets_dropdown.options = filtered_subjets
        
    else:
        
        area_dropdown.disabled=True
        filtered_subjets = data[data['NIVEL'] == selected_grade]['ASIGNATURA'].unique().tolist()
        subjets_dropdown.options = filtered_subjets

# Observar cambios en la lista despegable de niveles y ejecutar la función "update_subjets_dropdown" para actualizar la lista de asignaturas
grade_dropdown.observe(update_subjets_dropdown, names='value')

# Observar cambios en la Lista despegable de areas y ejecutar la función "update_subjets_dropdown" para actualizar la lista de asignaturas
area_dropdown.observe(update_subjets_dropdown, names='value')

# Actualización inicial de las opciones de asignaturas
update_subjets_dropdown()


In [4]:
# Organización de los controles interactivos mediante VBox, HBox  y layaout

# Crear el HBox con la alineación deseada
hbox_layout = Layout(justify_content='space-between')  # Centrar
vbox_layout = Layout(align_items = 'baseline',height='200%')
# Organizar en forma horizontal y vertical los controles interactivos de nivel, área, asignatura, y botones
widgets_layout = VBox([HBox([grade_dropdown, subjets_dropdown, area_dropdown],layout=hbox_layout),HBox([expand_buttons,select_date_multiple])],layout=vbox_layout)



In [5]:
widgets_layout

In [6]:
# Vincular los valores de los controles interactivos a la función "filtros_graficos" para poder graficar, usando el modo "interactive_output"
output = widgets.interactive_output(filtros_graficos, {
        'level_option': grade_dropdown,
        'subjets_option': subjets_dropdown,
        'area_option': area_dropdown,
        'expand_option': expand_buttons,
        'select_date' : select_date_multiple,
    })

In [8]:
# Mostrar los controles interactivos y el gráfico
display(widgets_layout, output)

Output()